# リアルタイム推論サービスを作成する

予測モデルのトレーニング後、クライアントが新しいデータから予測を取得するために使用できるリアルタイム サービスとしてモデルをデプロイできます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## モデルをトレーニングして登録する

それでは、モデルをトレーニングして登録しましょう。

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# ワークスペースで Azure 実験を作成する
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# トレーニング済みモデルを保存する
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 実行を完了する
run.complete()

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## モデルを Web サービスとして公開する

糖尿病の可能性に基づいて患者を分類する機械学習モデルをトレーニングし、登録しました。このモデルは、糖尿病の臨床検査を受ける必要があるとリスクがあると考えられる患者のみが必要な医師の手術などの運用環境で使用できます。このシナリオをサポートするには、モデルを Web サービスとしてデプロイします。

まず、ワークスペースに登録したモデルを決定しましょう。

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

それでは、デプロイしたいモデルを取得しましょう。既定では、モデル名を指定すると、最新バージョンが返されます。

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

このモデルをホストする Web サービスを作成しますが、これにはコードと構成ファイルが必要です。そのため、それらのフォルダーを作成してみましょう。

In [ ]:
import os

folder_name = 'diabetes_service'

# Web サービス ファイル用フォルダーを作成する
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# スコアリング スクリプトのパスを設定する
script_file = os.path.join(experiment_folder,"score_diabetes.py")

モデルをデプロイする Web サービスでは、入力データを読み込み、ワークスペースからモデルを取得し、予測を生成して返すために、Python コードが必要になります。このコードは、Web サービスにデプロイされる*エントリ スクリプト* (頻繁に*スコアリング スクリプト*と呼ばれます) に保存します。

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# サービスの読み込み時に呼び出される
def init():
    global model
    # デプロイ済みのモデル ファイルへのパスを取得して読み込む
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# 要求の受信時に呼び出される
def run(raw_data):
    # 入力データを numpy 配列として取得する
    data = np.array(json.loads(raw_data)['data'])
    # モデルから予測を取得する
    predictions = model.predict(data)
    # 各予測に対応するクラス名を取得する (0 または 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # 予測を JSON 形式で返す
    return json.dumps(predicted_classes)

Web サービスはコンテナーでホストされ、コンテナーは初期化されるときに必要な Python 依存関係をインストールする必要があります。この場合、スコアリング コードには **Scikit-learn** が必要なので、コンテナー ホストに環境にインストールするよう指示する .yml ファイルを作成します。

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# モデルの依存関係を追加する (AzureML の既定値は既に含まれています)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# 環境設定を .yml ファイルとして保存する
env_file = os.path.join(experiment_folder,"diabetes_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# .yml ファイルを印刷する
with open(env_file,"r") as f:
    print(f.read())

これでデプロイする準備ができました。コンテナーに **diabetes-service** という名前のサービスをデプロイします。デプロイ プロセスには、次のステップが含まれます。

1.モデルの読み込みと使用に必要なスコアリング ファイルと環境ファイルを含む推論構成を定義します。
2.サービスをホストする実行環境を定義するデプロイメント構成を定義します。この場合、Azure Container Instances。
3.モデルを Web サービスとしてデプロイする
4.デプロイされたサービスの状態を確認します。

> **詳細情報**: モデル デプロイ、ターゲット実行環境のオプションの詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)を参照してください。

デプロイは、最初にコンテナー イメージを作成するプロセスを実行し、そのイメージに基づいて Web サービスを作成するプロセスを実行するため、時間がかかります。デプロイが正常に完了すると、**正常**な状態が表示されます。

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# スコアリング環境を構成する
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

うまくいけば、デプロイが成功し、**正常**な状態を確認できます。確認できない場合は、次のコードを使用して、トラブルシューティングに役立つサービス ログを取得できます。

In [ ]:
print(service.get_logs())

# 変更を行って再デプロイする必要がある場合は、次のコードを使用して異常なサービスを削除することが必要となる可能性があります。
#service.delete()

[Azure Machine Learning Studio](https://ml.azure.com) でワークスペースを確認し、ワークスペースにデプロイされたサービスを示す**エンドポイント**ページを表示します。

次のコードを実行して、ワークスペース内の Web サービスの名前を取得することもできます。

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## Web サービスを使用する

サービスをデプロイしたら、クライアント アプリケーションからサービスを使用できます。

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# JSON ドキュメントでシリアル化可能なリストに配列を変換する
input_json = json.dumps({"data": x_new})

# 入力データを渡して Web サービスを呼び出す (Web サービスはバイナリ形式のデータも受け入れます)
predictions = service.run(input_data = input_json)

# 予測されたクラスを取得する - それは最初の (そして唯一の) クラスになります。
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

また、複数の患者の観察をサービスに送信し、それぞれの予測を取得することもできます。

In [ ]:
import json

# 今回の入力は、2 つの特徴配列のひとつです。
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# JSON ドキュメント内のシリアル化可能なリストに配列を変換する
input_json = json.dumps({"data": x_new})

# Web サービスを呼び出して入力データを渡す
predictions = service.run(input_data = input_json)

# 予測されたクラスを取得する
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

上記のコードでは、Azure Machine Learning SDK を使用してコンテナー化された Web サービスに接続し、それを使用して糖尿病分類モデルから予測を生成しています。運用環境では、Azure Machine Learning SDK を使用せず、単に Web サービスに HTTP 要求を行うビジネス アプリケーションによってモデルが使用される可能性があります。

これらのアプリケーションが要求を送信する必要がある URL を決定しましょう。

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

エンドポイント URI がわかったので、アプリケーションは HTTP 要求を行い、患者データを JSON 形式で送信し、予測されたクラスを受け取ることができます。

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# JSON ドキュメントでシリアル化可能なリストに配列を変換する
input_json = json.dumps({"data": x_new})

# コンテンツ タイプを設定する
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

認証を必要としない Azure Container Instances (ACI) サービスとして Web サービスをデプロイしました。これは開発とテストには適していますが、運用環境では Azure Kubernetes Service (AKS) クラスターへのデプロイとトークンベースの認証の有効化を検討する必要があります。これには、**Authorization** ヘッダーを含める REST 要求が必要です。

## サービスの削除

サービスが不要になった場合は、不要な料金が発生しないように削除する必要があります。

In [ ]:
service.delete()
print ('Service deleted.')

モデルをサービスとして公開する方法の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)を参照してください。